### Import Required Libraries and Set Up Environment Variables

In [66]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')
type(NASA_API_KEY)

str

### CME Data

In [ ]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs: (Coronal Mass Ejections)
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
url = f"{base_url}CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# print(url)


In [68]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(url)

# print(cme_response.content)

In [69]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [70]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [105]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[["activityID", "startTime", "linkedEvents"]]
cme_df.head()
cme_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5524 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activityID    5524 non-null   object
 1   startTime     5524 non-null   object
 2   linkedEvents  1023 non-null   object
dtypes: object(3)
memory usage: 129.6+ KB


In [103]:
print(cme_df["startTime"].min())
print(cme_df["startTime"].max())

2013-05-01T03:12Z
2024-05-01T23:16Z


In [96]:
# Notice that the linkedEvents column allows us to identify the corresponding GST (Geomagnetic Storm) for each CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=["linkedEvents"])
cme_df.head()
cme_df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1023 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   activityID    1023 non-null   object
 1   startTime     1023 non-null   object
 2   linkedEvents  1023 non-null   object
dtypes: object(3)
memory usage: 32.0+ KB


In [73]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index, row in cme_df.iterrows():
    # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
    for event in row["linkedEvents"]:
        expanded_rows.append({
            "activityID": row["activityID"],
            "startTime": row["startTime"],
            "linkedEvents": event
        })

# Create a new DataFrame from the expanded rows
cme_expanded_df = pd.DataFrame(expanded_rows)
cme_expanded_df.head()


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [74]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(linkedEvent):
        try:
                return linkedEvent["activityID"]
        except:
                # Log the error or print it for debugging
                print(f"Error with {linkedEvent}")
                return None

# verify that the function works as expected
print(extract_activityID_from_dict(cme_df.iloc[0]["linkedEvents"][0]))




2013-05-04T04:52:00-IPS-001


In [75]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_expanded_df.loc[:, "GST_ActivityID"] = cme_expanded_df["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
cme_expanded_df.head()


,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [76]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_expanded_df = cme_expanded_df.dropna(subset=["GST_ActivityID"])


In [77]:
# print out the datatype of each column in this DataFrame:
print(cme_expanded_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvents    1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB
None


In [78]:
# Convert the 'GST_ActivityID' column to string format 
cme_expanded_df["GST_ActivityID"] = cme_expanded_df["GST_ActivityID"].astype(str)

# Convert startTime to datetime format  
cme_expanded_df["startTime"] = pd.to_datetime(cme_expanded_df["startTime"])

# Rename startTime to startTime_CME and activityID to cmeID
cme_expanded_df = cme_expanded_df.rename(columns={"startTime": "startTime_CME", "activityID": "cmeID"})

# Drop linkedEvents
cme_expanded_df = cme_expanded_df.drop(columns=["linkedEvents"])

# Verify that all steps were executed correctly
cme_expanded_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 40.3+ KB


In [79]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_expanded_df = cme_expanded_df[cme_expanded_df["GST_ActivityID"].str.contains("GST")]
print(cme_expanded_df.head())


                           cmeID             startTime_CME  \
21   2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
48   2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00   
90   2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00   
148  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00   
151  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00   

                  GST_ActivityID  
21   2013-06-07T03:00:00-GST-001  
48   2013-10-02T03:00:00-GST-001  
90   2013-12-08T00:00:00-GST-001  
148  2014-02-19T03:00:00-GST-001  
151  2014-02-20T03:00:00-GST-001  


### GST Data

In [80]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
url = f"{base_url}GST?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [81]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(url)


In [82]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))


{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [83]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: gstID, startTime, linkedEvents
gst_df = gst_df[["gstID", "startTime", "linkedEvents"]]


In [84]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=["linkedEvents"])
gst_df.head()


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [85]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode("linkedEvents").reset_index(drop=True).dropna(subset=["linkedEvents"])


In [86]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df.loc[:, "CME_ActivityID"] = gst_df["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))


# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df.dropna(subset=["CME_ActivityID"])

print(gst_df.head())


                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents               CME_ActivityID  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  2013-05-31T15:45:00-HSS-001  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  2013-06-02T20:24:00-CME-001  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  2013-09-29T22:40:00-CME-001  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  2013-10-02T01:54:00-IPS-001  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  2013-10-02T02:47:00-MPC-001  


In [87]:
# Convert the 'CME_ActivityID' column to string format 
gst_df["CME_ActivityID"] = gst_df["CME_ActivityID"].astype(str)

# Convert the 'gstID' column to string format 
gst_df["gstID"] = gst_df["gstID"].astype(str)

# Convert startTime to datetime format  
gst_df["startTime"] = pd.to_datetime(gst_df["startTime"])

# Rename startTime to startTime_GST 
gst_df = gst_df.rename(columns={"startTime": "startTime_GST"})

# Drop linkedEvents
gst_df = gst_df.drop(columns=["linkedEvents"])

# Verify that all steps were executed correctly
gst_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.9+ KB


In [88]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df["CME_ActivityID"].str.contains("CME")]
print(gst_df.head())


                         gstID             startTime_GST  \
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID  
1  2013-06-02T20:24:00-CME-001  
2  2013-09-29T22:40:00-CME-001  
5  2013-12-04T23:12:00-CME-001  
7  2014-02-16T14:15:00-CME-001  
9  2014-02-18T01:25:00-CME-001  


### Merge both datatsets

In [89]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    gst_df, 
    cme_expanded_df, 
    left_on=['gstID', 'CME_ActivityID'], 
    right_on=['GST_ActivityID', 'cmeID'], 
    how='inner'
)

print(merged_df.head())


                         gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001  
2 2013-12-04 23:12:00+00:00  2013-12-08T00:00:00-GST-001  
3 2014-02-16 14:15:00+00

In [90]:
# Verify that the new DataFrame has the same number of rows as cme and gst
# gst_df.info()
# print("\n")
# cme_expanded_df.info()
# print("\n")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     object             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
 3   cmeID           61 non-null     object             
 4   startTime_CME   61 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](2), object(4)
memory usage: 3.0+ KB


### Computing the time it takes for a CME to cause a GST

In [91]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df["timeDiff"] = merged_df["startTime_GST"] - merged_df["startTime_CME"]
print(merged_df.head())


                         gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID        timeDiff  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001 4 days 06:36:00  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001 2 days 04:20:00  
2 2013-12-04 23:12:00+00:00  2013-1

In [92]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
print(merged_df["timeDiff"].describe())



count                           61
mean     2 days 23:29:26.557377049
std      0 days 23:53:09.336914240
min                1 days 08:36:00
25%                2 days 04:00:00
50%                2 days 19:51:00
75%                3 days 13:35:00
max                6 days 03:00:00
Name: timeDiff, dtype: object


### Exporting data in csv format

In [93]:
# Export data to CSV without the index
merged_df.to_csv("cme_gst.csv", index=False)


In [18]:
# I want to share this project with my team, so I need to print all dependencies and their versions
# I will use the pip freeze command to generate a requirements.txt file
# I will then print the contents of the file to the console
# import glob
# import os
# from re import T
# os.system("pip freeze > requirements.txt")
# os.system("cat requirements.txt")

# I will also print the versions of the libraries that I used in this project
# I will use the watermark library to print the versions of the libraries
# I will print the versions of all libraries
# I will also print the versions of the libraries that I used in this project

# print my python version, and print all my library versions
import sys
print(sys.version)

# print the versions of the libraries that I used in this project
import pandas as pd
import requests
import json
print(f"pandas version: {pd.__version__}")
print(f"requests version: {requests.__version__}")
print(f"json version: {json.__version__}")

# print all versions of packages and libraries I used in this project
import os
os.system("pip freeze")
# output to a file
os.system("pip freeze > requirements.txt")
# print the contents of the file
os.system("cat requirements.txt")







3.10.14 (main, May  6 2024, 14:42:37) [Clang 14.0.6 ]
pandas version: 2.2.2
requests version: 2.32.3
json version: 2.0.9
aiobotocore @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_1cl06d5vjc/croot/aiobotocore_1714464399334/work
aiohttp @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_1bydo4860s/croot/aiohttp_1715108783113/work
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
alabaster @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_39uesgct45/croot/alabaster_1718201495024/work
altair @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_a8x4081_4h/croot/altair_1687526044471/work
anyio @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_a17a7759g2/croot/anyio_1706220182417/work
appdirs==1.4.4
applaunchservices @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_1fiu9w6r9i/croots/

0

ourpy_1700583585236/work
cookiecutter @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_38jzqhs2jm/croot/cookiecutter_1711059824217/work
cryptography @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_31zgxm62w8/croot/cryptography_1714660690857/work
cssselect @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_47oh46v5h0/croot/cssselect_1707339886455/work
cycler @ file:///tmp/build/80754af9/cycler_1637851556182/work
cytoolz @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_f0etqooaak/croot/cytoolz_1701723613874/work
dask @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_48n3trhjfl/croot/dask-core_1715838604028/work
dask-expr @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_59fc337w7n/croot/dask-expr_1715846452490/work
datashader @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_90qkya6her/croot/datashader_1718121506841/work
debugpy @ file:///private/

0754af9/snowballstemmer_1637937080595/work
sortedcontainers @ file:///tmp/build/80754af9/sortedcontainers_1623949099177/work
soupsieve @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_9798xzs_03/croot/soupsieve_1696347567192/work
Sphinx @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_50hy2c49dh/croot/sphinx_1718275395179/work
sphinxcontrib-applehelp @ file:///home/ktietz/src/ci/sphinxcontrib-applehelp_1611920841464/work
sphinxcontrib-devhelp @ file:///home/ktietz/src/ci/sphinxcontrib-devhelp_1611920923094/work
sphinxcontrib-htmlhelp @ file:///tmp/build/80754af9/sphinxcontrib-htmlhelp_1623945626792/work
sphinxcontrib-jsmath @ file:///home/ktietz/src/ci/sphinxcontrib-jsmath_1611920942228/work
sphinxcontrib-qthelp @ file:///home/ktietz/src/ci/sphinxcontrib-qthelp_1611921055322/work
sphinxcontrib-serializinghtml @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_c0gy7ylkec/croot/sphinxcontrib-serializinghtml_1718201677157/work
s